# Enter Trip Information

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Enter a trip name (a unique identifier for the specified points of interest)
TRIP_NAME = "east"

# Enter your points of interest in the list below (include your starting location)
MY_POINTS_OF_INTEREST = [
    "San Francisco, California", # Starting Location
    "Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA", # Yosemite NP
    "D L Bliss State Park, California 89, South Lake Tahoe, CA", # Lake Tahoe
    "Zion National Park Visitor Center, Zion – Mount Carmel Highway, Hurricane, UT", # Zion NP
    "Arches National Park Visitor Center, Moab, UT", # Arches NP
    "Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ", # Monument Valley
    "Island in the Sky Visitor Center, Grand View Point Road, Moab, UT", # Canyonlands NP
    "Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT", # Bryce Canyon NP
    "Moro Rock Trail, California", # Sequoia NP
    "Horseshoe Bend Parking Lot, Page, AZ", # Horseshoe Bend Trail
    "Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ", # Grand Canyon NP
    "Calf Creek Campground, Boulder, UT", # Grand Staircase-Escalante NM
    "Red Cliffs Recreation Area, Unnamed Road, Washington, UT", # Red Cliffs Recreation Nature Trail
    "Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT", # Natural Bridges NM
    "Kanarra Creek Trailhead, Kanarraville, UT", # Kanarra Creek Canyon Trail
    "San Simeon, CA", # South Big Sur Drive
    "Big Sur, CA", # Middle Big Sur Drive
    "Carmel-by-the-Sea, CA", # North Big Sur Drive
    "Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA", # Saratoga Gap Trail
]

# Collect Driving Distance/Duration Information

In [3]:
from src.data_collection import *
from config import GOOGLE_MAPS_API_KEY


# Determine distance and duration filename based on specified trip name
distance_duration_filename = "data/my_{}_points_of_interest_distance_duration.csv".format(TRIP_NAME)

# Try to create a distance and duration df containing all my points of interest from the filename
try:
    
    distance_duration_df = pd.read_csv(distance_duration_filename, index_col=0)

    # Create list of unique points of interest from df
    df_points_of_interest = set(pd.unique(distance_duration_df[['Venue 1', 'Venue 2']].values.ravel('K')))

    # Check if missing one or more of my points of interest in df
    if not set(MY_POINTS_OF_INTEREST).issubset(df_points_of_interest):
        
        raise Exception("Missing one or more of my points of interest in '{}'".format(distance_duration_filename))

# Create a distance and duration df with all my points of interest and save to the specified filename
except (FileNotFoundError, Exception) as e:
        
    # Query Google Maps API for one-way driving distances and durations
    distance_duration_data = query_gmaps_api_for_one_way_driving_distance_and_duration(MY_POINTS_OF_INTEREST, GOOGLE_MAPS_API_KEY)

    # Create DataFrame of one-way distances and durations
    distance_duration_df = create_distance_and_duration_df(distance_duration_data)

    # Save DataFrame to CSV
    distance_duration_df.to_csv(distance_duration_filename)
    
# Preview distance and duration df
distance_duration_df.head().sort_values('Distance (mi)', ascending=False)

,Venue 1,Venue 2,Distance (mi),Duration (s),Duration (hhmm)
3,"San Francisco, California","Arches National Park Visitor Center, Moab, UT",963,51219,14:13
4,"San Francisco, California","Monument Valley Navajo Tribal Park, Main Monum...",941,52195,14:29
2,"San Francisco, California","Zion National Park Visitor Center, Zion – Moun...",727,40019,11:06
1,"San Francisco, California","D L Bliss State Park, California 89, South Lak...",197,12542,3:29
0,"San Francisco, California","Yosemite Valley Visitor Center, Village Drive,...",191,14181,3:56


## *Optional : Display Full Name Squareform Distance/Duration Matrices*

In [3]:
# Add reverse travel information (B to A not just A to B) to distance and duration df
_df = add_reverse_travel_information_to_distance_duration_df(distance_duration_df)

# Create squareform matrices
distance_matrix = _df.pivot(index='Venue 1', columns='Venue 2', values='Distance (mi)').fillna(0).astype(int)
duration_matrix = _df.pivot(index='Venue 1', columns='Venue 2', values='Duration (s)').fillna(0).astype(int)
duration_matrix_hhmm = _df.pivot(index='Venue 1', columns='Venue 2', values='Duration (hhmm)').fillna("0:00")

In [ ]:
# # Display distance matrix
# display(distance_matrix)

In [ ]:
# # Display duration matrix
# display(duration_matrix)

In [ ]:
# Display duration hhmm matrix
display(duration_matrix_hhmm)

## *Optional : Display Integer Name Squareform Distance/Duration Matrices*

In [4]:
# Add reverse travel information (B to A not just A to B) to distance and duration df
_df = add_reverse_travel_information_to_distance_duration_df(distance_duration_df)

# Convert venue columns to categorical type and create categorical code columns
_df['Venue 1'] = _df['Venue 1'].astype('category')
_df['Venue 1 Codes'] = _df['Venue 1'].cat.codes
_df['Venue 2'] = pd.Categorical(_df['Venue 2'], categories=_df['Venue 1'].cat.categories)
_df['Venue 2 Codes'] = _df['Venue 2'].cat.codes

# Create squareform matrices with codes
distance_matrix = _df.pivot(index='Venue 1 Codes', columns='Venue 2 Codes', values='Distance (mi)').fillna(0).astype(int)
duration_matrix = _df.pivot(index='Venue 1 Codes', columns='Venue 2 Codes', values='Duration (s)').fillna(0).astype(int)
duration_matrix_hhmm = _df.pivot(index='Venue 1 Codes', columns='Venue 2 Codes', values='Duration (hhmm)').fillna("0:00")

In [ ]:
# # Preview new columns
# _df.sample(5)

In [ ]:
# # Display distance matrix with code mappings
# display(distance_matrix)

# # Print dict of code: cat mappings for reference
# _ = dict(enumerate(_df['Venue 1'].cat.categories))
# for k, v in _.items():
#     print(k, ":", v)

In [ ]:
# # Display duration matrix with code mappings
# display(duration_matrix)

# # Print dict of code: cat mappings for reference
# _ = dict(enumerate(_df['Venue 1'].cat.categories))
# for k, v in _.items():
#     print(k, ":", v)

In [ ]:
# Display duration matrix hhmm with code mappings
display(duration_matrix_hhmm)

# Print dict of code: cat mappings for reference
_ = dict(enumerate(_df['Venue 1'].cat.categories))
for k, v in _.items():
    print(k, ":", v)

# Optimize Road Trip via Genetic Algorithm

Thank you to [Randal S. Olson](http://www.randalolson.com/) for the genetic algorithm code below (sourced from [this notebook](https://github.com/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/optimal-road-trip/Computing%20the%20optimal%20road%20trip%20across%20the%20U.S..ipynb) and further explained in [this blog post](http://www.randalolson.com/2015/03/08/computing-the-optimal-road-trip-across-the-u-s/)). All the credit for the code goes to him with a few minor adjustments made by me.

In [65]:
from src.genetic_algorithm import *

# Run the genetic algorithm
results = run_genetic_algorithm(MY_POINTS_OF_INTEREST, distance_duration_filename, generations=5000, population_size=100)

# Update HTML file to display correct results
update_results_html_file("./src/results_template.html", "./results/results.html", results)

Generation 0 best: 5704 | Unique genomes: 100
('Arches National Park Visitor Center, Moab, UT', 'Island in the Sky Visitor Center, Grand View Point Road, Moab, UT', 'Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT', 'Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT', 'Red Cliffs Recreation Area, Unnamed Road, Washington, UT', 'Zion National Park Visitor Center, Zion – Mount Carmel Highway, Hurricane, UT', 'San Simeon, CA', 'Horseshoe Bend Parking Lot, Page, AZ', 'Kanarra Creek Trailhead, Kanarraville, UT', 'Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ', 'Calf Creek Campground, Boulder, UT', 'Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ', 'D L Bliss State Park, California 89, South Lake Tahoe, CA', 'Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA', 'San Francisco, California', 'Carmel-by-the-Sea, CA', 'Yosemite Valley Visitor Center, Village Drive, Yosemite Valley

Generation 4000 best: 2922 | Unique genomes: 91
('Kanarra Creek Trailhead, Kanarraville, UT', 'D L Bliss State Park, California 89, South Lake Tahoe, CA', 'San Francisco, California', 'Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA', 'Carmel-by-the-Sea, CA', 'Big Sur, CA', 'San Simeon, CA', 'Moro Rock Trail, California', 'Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA', 'Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT', 'Calf Creek Campground, Boulder, UT', 'Island in the Sky Visitor Center, Grand View Point Road, Moab, UT', 'Arches National Park Visitor Center, Moab, UT', 'Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT', 'Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ', 'Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ', 'Horseshoe Bend Parking Lot, Page, AZ', 'Zion National Park Visitor Center, Zion – Mount Carmel Highway, Hurricane, UT', 'Red 

In [66]:
# View map of results (map opens in new tab)
!open ./results/results.html

In [67]:
# View distance and duration df of results
results_df = create_results_df(distance_duration_filename, results)
print("Total driving distance {} miles ...".format(results_df['Distance (mi)'].sum()))
display(results_df)

Total driving distance 2922 miles ...


,Venue 1,Venue 2,Duration (hhmm),Distance (mi)
0,"Kanarra Creek Trailhead, Kanarraville, UT","D L Bliss State Park, California 89, South Lak...",9:06,540
1,"D L Bliss State Park, California 89, South Lak...","San Francisco, California",3:29,197
2,"San Francisco, California","Castle Rock Entrance Station Parking Lot, Unna...",1:09,55
3,"Castle Rock Entrance Station Parking Lot, Unna...","Carmel-by-the-Sea, CA",1:25,70
4,"Carmel-by-the-Sea, CA","Big Sur, CA",1:03,26
5,"Big Sur, CA","San Simeon, CA",2:21,80
6,"San Simeon, CA","Moro Rock Trail, California",3:59,196
7,"Moro Rock Trail, California","Yosemite Valley Visitor Center, Village Drive,...",4:29,194
8,"Yosemite Valley Visitor Center, Village Drive,...","Bryce Canyon National Park Visitor Center, Uta...",10:03,583
9,"Bryce Canyon National Park Visitor Center, Uta...","Calf Creek Campground, Boulder, UT",1:21,66


## *Optional : Save Results Map and DataFrame Shown Above*